In [1]:
import torch
from datasets import load_dataset
from datasets import load_from_disk

# from d2l import torch as d2l


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 定义数据集
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, split):
        # dataset = load_dataset('lansinuote/ChnSentiCorp', keep_in_memory=True)
        dataset = load_from_disk('./data/ChnSentiCorp')
        
        def f(data):
            return len(data['text']) > 30
        
        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        return text

In [3]:
dataset = Dataset('train')

len(dataset) # , dataset[0]

Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/validation/cache-cf45964edee402a8.arrow
Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/test/cache-c6f7400aef16ddba.arrow
Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/train/cache-478819d08c52879a.arrow


3

In [4]:
from transformers import BertTokenizer
from transformers import BertModel


# 加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

print(token)


# 加载预训练模型
bert = BertModel.from_pretrained('bert-base-chinese').to(device)

print('加载完毕..')

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def collate_fn(data):
    
    # print(data)
    # print('data长度: ', len(data))
    # 编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=data,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=30,
                                   return_tensors='pt',
                                   return_length=True)

    # print('编码后的data打印: ', data.keys())
    
    # input_ids:编码之后的数字
    # attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']

    # 把第15个词固定替换为mask
    # print('第15个词: %s' % input_ids[:, 15])
    
    # 这里直接使用了编码后的数据作为真实预测值
    labels = input_ids[:, 15].reshape(-1).clone()
    input_ids[:, 15] = token.get_vocab()[token.mask_token]

    # print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels

## 数据加载器

In [6]:
# 数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset['train'],
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True
                                    )


for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    print('当前loader数据量: ', len(loader))
    print('解码input_ids: ', token.decode(input_ids[0]))
    print('labels: ', token.decode(labels[0]))
    print('参数打印: ', input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)
    print("")
    if i >= 1:
        break

当前loader数据量:  574
解码input_ids:  [CLS] 开 箱 即 发 现 [UNK] 口 一 个 是 坏 的 ， 触 [MASK] 板 左 键 也 是 坏 的 ， 惠 普 的 质 量 [SEP]
labels:  摸
参数打印:  torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])

当前loader数据量:  574
解码input_ids:  [CLS] 外 观 很 炫 ， 同 事 们 看 了 都 很 羡 慕 [MASK] 性 能 还 不 错 的 ， 单 从 配 置 来 看 [SEP]
labels:  ！
参数打印:  torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])



## 定义下游任务模型

In [7]:
# 定义下游任务模型
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.decoder = torch.nn.Linear(768, token.vocab_size, bias=False)
        self.bias = torch.nn.Parameter(torch.zeros(token.vocab_size))
        self.decoder.bias = self.bias

    def forward(self, input_ids, attention_mask, token_type_ids):
        # pretrained = pretrained.to(device)
        with torch.no_grad():
            out = bert(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)
            pass
        
        out = self.decoder(out.last_hidden_state[:, 15])
        return out


In [8]:

# 加载保存的模型
model_save_path = 'chinese_full_vacant_mission_2023_4_10.pt'
model = Model()
model.load_state_dict(torch.load(model_save_path))

# 模型转移到GPU上
model.to(device)

# list(model.parameters())[0].device  # device(type='cuda', index=0)

Model(
  (decoder): Linear(in_features=768, out_features=21128, bias=True)
)

## 模型测试

In [18]:
#测试
def test_calculate(stop_num=20):
    
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=dataset['test'],
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)
    for i, (input_ids, 
            attention_mask, 
            token_type_ids, labels) in enumerate(loader_test):
        if i >= stop_num:
            break

        with torch.no_grad():
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            labels = labels.to(device)
            
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
            pass
        
        out = out.cpu()
        labels = labels.cpu()
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)
        
        correct += accuracy
        total += 1
        
        y = token.decode(labels[0])
        y_hat = token.decode(out[0])
        
        if y != y_hat:
            print('序: %s 输入内容: %s' % (i, token.decode(input_ids[0])))
            print('[ERROR] label: %s, \t y_hat: %s' %(y, y_hat))
            print("")
        else:
            pass
        
    print('total: %s acc: %.2f' % (total, correct / total))
    pass

In [19]:
# 预测
test_calculate(stop_num=50)

序: 0 输入内容: [CLS] 第 一 次 买 的 拉 拉 升 职 记 ， 三 天 到 [MASK] （ 我 住 北 京 三 环 边 上 ） ， 还 比 [SEP]
[ERROR] label: 货, 	 y_hat: 了

序: 2 输入内容: [CLS] 书 已 收 到 ， 貌 似 觉 得 字 有 点 小 ， [MASK] ， 可 惜 了 书 里 的 内 容 几 乎 和 网 [SEP]
[ERROR] label: 哎, 	 y_hat: 但

序: 5 输入内容: [CLS] 转 轴 设 计 不 好 ， 合 上 屏 幕 后 后 面 [MASK] 隙 很 大 ， 手 一 抓 就 变 形 了 ， 只 [SEP]
[ERROR] label: 缝, 	 y_hat: 间

序: 7 输入内容: [CLS] 酒 店 房 间 很 大 ， 这 是 最 大 的 优 点 [MASK] 设 施 不 是 很 新 ， 楼 道 的 空 调 太 [SEP]
[ERROR] label: 。, 	 y_hat: ，

序: 13 输入内容: [CLS] ： 配 置 不 错 ， 奔 腾 双 核 感 觉 也 不 [MASK] ~ 价 格 合 适 不 足 ： 装 系 统 比 较 [SEP]
[ERROR] label: 差, 	 y_hat: 错

序: 17 输入内容: [CLS] 性 价 比 非 常 好 的 一 款 ， 推 荐 购 买 [MASK] 自 己 又 添 了 一 根 [UNK] 的 条 子 ， 用 [SEP]
[ERROR] label: ！, 	 y_hat: ，

序: 18 输入内容: [CLS] 原 本 在 网 上 订 了 两 个 套 房 ， 入 住 [MASK] ， 携 程 还 给 我 打 电 话 问 是 否 只 [SEP]
[ERROR] label: 后, 	 y_hat: 了

序: 20 输入内容: [CLS] 没 买 的 就 不 用 买 了 ， 到 新 浪 网 上 [MASK] 书 频 道 看 一 眼 足 够 了 。 毕 大 夫 [SEP]
[ERROR] label: 读, 	 y_hat: 的

序: 22 输入内容: [CLS] 买 之 前 也 没 见 过 这 本 书, 听 他 们 [MASK] 的 天 花 乱 坠, 翻 了 几 页 就 够 了 [SE